In [1]:
! pip install -q pyspark==3.2.0 spark-nlp==3.4.2. #requirements για να το τρέξουμε στο google colab

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 142 kB 15.5 MB/s 
     |████████████████████████████████| 198 kB 64.0 MB/s 


In [2]:

# ξεκινάμε τη sparknlp

import sparknlp

spark = sparknlp.start(gpu = True, spark32=True) 
# for GPU training >> sparknlp.start(gpu = True) colab άρα είναι εφικτό
 # for Spark 2.3 =>> sparknlp.start(spark23 = True)

from sparknlp.base import *                  # απαραίτητα κυρίως για bert και embeddings, classifierdl
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import os

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version) #ένα τσεκ για τις εκδόσεις που εγκαταστήσαμε στο session μας 

spark


Spark NLP version 3.4.2
Apache Spark version: 3.2.0


In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *  # δε χρειαζόμαστε όλες τις sql συναρτήσεις της pyspark , αλλά εφόσον δεν επιβαρύνουν τα resources το αφήνουμε
from sparknlp.annotator import *
from sparknlp import *
from sparknlp.common import *
from sparknlp.training import *
df = spark.read.option("header","true") \
  .option("sep", ",") \
  .option("multiLine", "true") \
  .option("quote","\"") \
  .option("escape","\"")\
  .option("ignoreTrailingWhiteSpace", True) \
  .csv("job_postings.csv")

df = df.drop(*[c for c in df.columns if c not in {"description","fraudulent"}])  # το βήμα αυτό έχει εξηγηθεί και στο preprocess σε άλλο αρχείο
df.printSchema()                                                                  # θέλουμε μόνο description και fraudulent ως στήλες 

df.show(30)

root
 |-- description: string (nullable = true)
 |-- fraudulent: string (nullable = true)

+--------------------+----------+
|         description|fraudulent|
+--------------------+----------+
|Food52, a fast-gr...|         0|
|Organised - Focus...|         0|
|Our client, locat...|         0|
|THE COMPANY: ESRI...|         0|
|JOB TITLE: Itemiz...|         0|
|Job OverviewApex ...|         0|
|Your Responsibili...|         0|
|Who is Airenvy?He...|         0|
|Implementation/Co...|         0|
|The Customer Serv...|         0|
|Position : #URL_8...|         0|
|TransferWise is t...|         0|
|The Applications ...|         0|
|Event Industry In...|         0|
|Are you intereste...|         0|
|About Vault Drago...|         0|
|We are looking fo...|         0|
|Government fundin...|         0|
|Kettle is hiring ...|         0|
|Experienced Proce...|         0|
|IntelliBright is ...|         0|
|Want to be part o...|         0|
|The position repo...|         0|
|#URL_eda2500ddced...|   

In [ ]:
# πάμε να φτιάξουμε το pipeline για να χρησιμοποιήσουμε έναν bert sentence embedder, για λόγους χρόνου βάζουμε έναν small bert
# eεπειδή εδώ δε φαίνεται η ανάθεση διανυσμάτων στις λέξεις του description (προτάσεις εδώ με τον sentence embedder)
# έχουμε κάνει και έναν άλλον bert embedder με mbedder finisher για να φαίνονται και τα διανύσματα κάθε λέξης εκεί 


# μοντέλο το sent_small_bert_L8_128
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")

embeddingsSentence = BertSentenceEmbeddings().pretrained("sent_small_bert_L8_128", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings") 

classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("fraudulent")\
    .setMaxEpochs(7)\
    .setEnableOutputLogs(True)

bert_clf_pipeline = Pipeline(stages=[
    document_assembler,
    embeddingsSentence,
    classsifierdl
])

splits = df.randomSplit([0.8, 0.2])
train = splits[0]
test = splits[1]

b_clf_pipelineModel = bert_clf_pipeline.fit(train)
preds = b_clf_pipelineModel.transform(test)
preds_df = preds.select('fraudulent','description',"class.result").toPandas()
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])
from sklearn.metrics import classification_report

print(classification_report(preds_df['fraudulent'], preds_df['result']))

sent_small_bert_L8_128 download started this may take some time.
Approximate size to download 20.5 MB
[OK!]
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      3429
           1       0.00      0.00      0.00       185

    accuracy                           0.95      3614
   macro avg       0.47      0.50      0.49      3614
weighted avg       0.90      0.95      0.92      3614



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
from pyspark.sql.functions import *
from sparknlp.base import *                           # ιδιες εισαγωγες απλα χρησιμοποιηθηκαν αρκετες φορες και για να μην ξανατρεχει απο την αρχη ο 
from sparknlp.annotator import *                      # bert και ξανα με το majordf 
from sklearn.metrics import classification_report, accuracy_score
# we will try oversampling on our dataset ( it is very imbalanced though since 95% are zeros on fraudulent col)

major_df = df.filter(col("fraudulent") == 0)
minor_df = df.filter(col("fraudulent") == 1)
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))
# duplicate the minority rows (ones)
a = range(ratio)
oversampled_df = minor_df.withColumn("dummy", explode(array([lit(x) for x in a]))).drop('dummy')

df = major_df.unionAll(oversampled_df)
# Maybe this gives us a more balanced dataset
df.groupBy('fraudulent').count().show()   # για να δούμε τώρα αν έφτιαξε το dataset , έφτιαξε αρκετά 
# δημιουργήσαμε νέους άσους(εγγραφές με fraudulent 1 ), ωστόσο τα αποτελέσματα θα είναι λίγο πλασματικά 
# μιας και απλά αυξήσαμε σχεδόν για να φτάσουν στα ίδια μεγέθη

ratio: 19
+----------+-----+
|fraudulent|count|
+----------+-----+
|         0|17014|
|         1|16454|
+----------+-----+



In [13]:

#OVERSAMPLED DATASET - BERT SENTENCE CLASSIFIERDL


document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")

embeddingsSentence = BertSentenceEmbeddings().pretrained("sent_small_bert_L8_128", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings") 

classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("fraudulent")\
    .setMaxEpochs(7)\
    .setEnableOutputLogs(True)

bert_clf_pipeline = Pipeline(stages=[
    document_assembler,
    embeddingsSentence,
    classsifierdl
])

splits = df.randomSplit([0.8, 0.2])
train = splits[0]
test = splits[1]

b_clf_pipelineModel = bert_clf_pipeline.fit(train)
preds = b_clf_pipelineModel.transform(test)
preds_df = preds.select('fraudulent','description',"class.result").toPandas()
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])
from sklearn.metrics import classification_report

print(classification_report(preds_df['fraudulent'], preds_df['result']))

sent_small_bert_L8_128 download started this may take some time.
Approximate size to download 20.5 MB
[OK!]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3408
           1       0.49      1.00      0.66      3287

    accuracy                           0.49      6695
   macro avg       0.25      0.50      0.33      6695
weighted avg       0.24      0.49      0.32      6695



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
